<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recommendation system.**


# I. Tổng quát 


---


Recommender System (RS) là một hệ thống khuyến nghị. Nhiệm vụ của RS là dự đoán mức độ quan tâm của người dùng với một sản phẩm nào đó thông qua các mô hình khuyến nghị thích hợp.

Cần quan tâm: users, items, ratings.
# II. Mô hình khuyến nghị


---


a. Content-based System

Khuyến nghị dựa trên đặc tính sản phẩm. Dễ thấy nhất là các hệ thống sử dụng loại, thể loại sản phẩm mà user tiêu thụ để khuyến nghị cho họ những sản phẩm có loại, thể loại tương tự. 

--> những sản phẩm không rõ phân loại, hay quá nhiều phân loại sẽ ảnh hưởng đến acuracy của mô hình.

b. Collaborative Filtering

Khuyến nghị dựa trên sự tương quan giữa users vs users, items vs items, hoặc cả 2. Ví dụ một nhóm user A, B, C thích item I và dữ liệu cho thấy B, C cũng thích item I2. Hệ thống sẽ khuyến nghị item I2 cho A vì có khả năng A cũng thích I2.

--> hoàn thiện hơn Content-based System.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import time

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [ ]:
from pandas import DataFrame

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/sample-datasets/ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
ratings.head(5)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
ratings.shape

(100836, 3)

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/sample-datasets/movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})

In [ ]:
movies.head(5)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [ ]:
movies.shape

(9742, 2)

In [ ]:
ratings_df = ratings.copy()
movies_df = movies.copy()

In [ ]:
df = pd.merge(ratings_df,movies_df,on='movieId')

In [ ]:
df.head(5)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [ ]:
df.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042562
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


In [ ]:
util_mat=df.pivot_table(index='userId',columns='title',values='rating')
util_mat.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
util_mat.loc[7,:].dropna()

title
2001: A Space Odyssey (1968)           4.0
A.I. Artificial Intelligence (2001)    4.5
Aladdin (1992)                         3.0
American Beauty (1999)                 4.0
Apocalypse Now (1979)                  4.0
                                      ... 
What Women Want (2000)                 4.0
Wild Wild West (1999)                  1.5
X-Men (2000)                           3.5
X-Men: The Last Stand (2006)           4.0
X2: X-Men United (2003)                4.0
Name: 7, Length: 152, dtype: float32

In [ ]:
user_util_matrix = util_mat.copy()
user_util_matrix = user_util_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)
user_util_matrix.head(5)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.000000,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.000000,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.4

In [ ]:
user_util_matrix.T.corr(method ='pearson')

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,1.264517e-03,5.525789e-04,0.048419,0.021847,-0.045497,-6.199668e-03,0.047013,1.950986e-02,-8.754091e-03,-0.056206,0.013266,5.938345e-02,0.021815,0.037209,0.009664,0.037997,0.043789,0.085249,6.699048e-02,0.011732,-0.004171,-0.023124,0.008417,0.001181,-1.351818e-02,4.530300e-02,0.001611,-0.006969,3.274415e-03,6.956396e-03,0.042510,0.012918,0.011187,2.150784e-02,2.652436e-02,-4.934852e-02,0.009979,-0.072666,-0.041753,...,-3.037625e-02,0.046056,1.408778e-02,4.071870e-02,1.232655e-03,-8.180505e-05,0.089396,-4.457347e-17,-1.050026e-01,0.019428,0.007481,9.521894e-03,-5.892850e-03,-1.114252e-02,1.002266e-02,0.011086,-4.307956e-02,-0.019135,-0.059246,0.095191,3.827620e-02,-5.950683e-02,0.082403,-0.006195,3.652910e-02,0.035349,0.102631,4.179380e-03,0.052614,0.069528,0.018127,-0.017172,-0.015221,-3.705874e-02,-2.912138e-02,0.012016,0.055261,0.075224,-2.571255e-02,0.010932
2,0.001265,1.000000e+00,-1.043295e-17,-0.017164,0.021796,-0.021051,-1.111357e-02,-0.048085,-1.350077e-17,3.011628e-03,-0.002745,0.000000,3.047562e-03,0.012518,-0.033280,-0.014771,-0.001263,-0.047825,-0.000106,6.099875e-04,-0.001949,-0.024828,-0.001169,-0.037418,0.016208,1.374669e-17,3.121718e-18,-0.004240,-0.019714,-3.399464e-02,2.180116e-17,-0.032900,0.008467,-0.008806,-4.549385e-18,-3.370691e-02,7.806388e-03,-0.042923,0.000000,-0.026103,...,-1.216991e-17,-0.006322,2.238276e-02,-5.331292e-02,1.613788e-17,-1.266533e-17,-0.029436,-1.854168e-17,3.784501e-17,-0.009902,-0.071276,-2.549879e-02,-1.193539e-03,-3.841119e-18,1.072661e-02,-0.012535,1.308529e-17,-0.048090,-0.035668,0.002703,2.482892e-03,-1.031647e-02,-0.026120,0.000338,-4.229444e-17,0.031263,0.006005,1.917656e-03,-0.000213,-0.000980,-0.050551,-0.031581,-0.001688,1.036375e-17,1.265390e-17,0.006226,-0.020504,-0.006001,-6.009099e-02,0.024999
3,0.000553,-1.043295e-17,1.000000e+00,-0.011260,-0.031539,0.004800,1.866590e-17,-0.032471,-8.931480e-18,-1.650113e-17,0.000000,0.000000,1.531215e-17,0.008453,-0.046046,-0.005142,-0.004690,-0.012322,-0.009560,-2.114475e-02,-0.022848,0.010239,-0.030150,-0.009327,-0.015357,9.094170e-18,-5.515577e-03,0.016249,-0.017537,6.391834e-17,-1.704942e-03,-0.045848,-0.030532,0.029048,-3.009660e-18,2.337270e-17,-9.364542e-18,-0.028985,0.028645,-0.003331,...,-2.542548e-02,-0.005930,4.996275e-17,1.459272e-18,1.067607e-17,-4.999971e-02,-0.037601,-1.226631e-17,-1.013572e-03,0.015241,-0.037817,4.090067e-17,1.696908e-17,-2.541105e-18,7.040272e-17,0.031381,-1.332141e-02,-0.014251,-0.024086,-0.028410,-1.811789e-17,-2.637889e-18,0.010920,-0.007823,-2.798003e-17,0.018377,-0.020662,8.434871e-19,0.000223,-0.004669,-0.004904,-0.016117,0.017749,6.856184e-18,-1.430628e-03,-0.037289,-0.007789,-0.013001,2.822380e-17,0.019550
4,0.048419,-1.716402e-02,-1.125978e-02,1.000000,-0.029620,0.013956,5.809139e-02,0.002065,-5.873604e-03,5.159032e-02,0.062453,-0.016056,-5.903304e-02,-0.058388,0.033399,-0.027573,-0.039002,-0.004090,0.009273,3.542258e-02,-0.005827,-0.006829,-0.014836,-0.008388,-0.008842,-1.729245e-02,1.700677e-02,0.014851,0.033426,9.596850e-04,-6.446310e-03,0.056834,0.011651,0.050031,4.037648e-03,-1.010066e-03,-1.558075e-02,0.022612,0.023640,-0.002802,...,5.153728e-03,-0.014396,-1.012133e-03,3.299309e-02,-2.610604e-02,-2.594862e-02,0.060230,1.406328e-17,-2.099000e-02,-0.078191,0.049113,-2.630610e-02,-3.076547e-02,-8.766061e-03,4.294414e-03,-0.016293,-4.037276e-02,0.007293,-0.008866,0.087103,-1.826945e-02,-1.676408e-02,0.057924,-0.007381,9.839639e-03,-0.001616,0.044671,-1.585653e-02,0.014862,0.023748,-0.037687,0.063122,0.027640,-1.378212e-02,4.003747e-02,0.020590,0.014628,-0.037569,-1.788358e-02,-0.000995
5,0.021847,2.179572e-02,-3.153

In [ ]:
# from sklearn.metrics import pairwise_distances
# from scipy.spatial.distance import cosine

# dist_out = 1-pairwise_distances(user_util_matrix.T, metric="cosine")
# dist_out

In [ ]:
copy_corr_mat = user_util_matrix.T.corr(method ='pearson')

In [ ]:
user7_movies = df[df['userId'] == 7]
user7_movies

,userId,movieId,rating,title
2,7,1,4.5,Toy Story (1995)
575,7,50,4.5,"Usual Suspects, The (1995)"
1570,7,260,5.0,Star Wars: Episode IV - A New Hope (1977)
2428,7,356,5.0,Forrest Gump (1994)
3190,7,480,5.0,Jurassic Park (1993)
...,...,...,...,...
39672,7,48997,2.5,Perfume: The Story of a Murderer (2006)
39680,7,49272,4.5,Casino Royale (2006)
39761,7,49278,3.5,Déjà Vu (Deja Vu) (2006)
39781,7,49286,0.5,"Holiday, The (2006)"


In [ ]:
util_mat.loc[7, :]

title
'71 (2014)                                  NaN
'Hellboy': The Seeds of Creation (2004)     NaN
'Round Midnight (1986)                      NaN
'Salem's Lot (2004)                         NaN
'Til There Was You (1997)                   NaN
                                             ..
eXistenZ (1999)                             NaN
xXx (2002)                                  NaN
xXx: State of the Union (2005)              NaN
¡Three Amigos! (1986)                       NaN
À nous la liberté (Freedom for Us) (1931)   NaN
Name: 7, Length: 9719, dtype: float32

In [ ]:
df[df['title'] == "'Hellboy': The Seeds of Creation (2004)"]

,userId,movieId,rating,title
97100,332,97757,4.0,'Hellboy': The Seeds of Creation (2004)


In [ ]:
users_who_rated_97757 = df[df['movieId'] == 1]['userId'].values
users_who_rated_97757[0:5]

array([ 1,  5,  7, 15, 17], dtype=int32)

In [ ]:
def find_user_neighbours(userId, users_who_rated_item, k=10):
  sims = []
  users = []
  for user in users_who_rated_item:
    sim = copy_corr_mat.loc[user, userId] 
    sims.append(sim)
    users.append(user)
  sims_df = pd.DataFrame({'user': users, 'similarity': sims}).set_index('user')
  sims_df.sort_values(by='similarity', ascending=False, inplace=True)
  closet_neighbours = sims_df[1:k]
  closet_neighbours
  return closet_neighbours

In [ ]:
users_who_rated_item = df[df['movieId'] == 1]['userId'].values
closet_neighbours = find_user_neighbours(7, users_who_rated_item)
closet_neighbours

,similarity
user,
434,0.137777
590,0.129029
220,0.125077
219,0.118220
605,0.108622
276,0.108187
596,0.107557
380,0.104129
63,0.099059


In [ ]:
def predict_rating(userId, movieId):
  sum_similarity = 0
  weighted_ratings = 0
  users_who_rated_item = df[df['movieId'] == movieId]['userId'].values
  closet_neighbours = find_user_neighbours(userId, users_who_rated_item)
  for user in closet_neighbours.index:
    weighted_ratings += closet_neighbours.loc[user] * df[ (df['userId'] == user) & (df['movieId'] == movieId) ]['rating'].sum()
    sum_similarity += closet_neighbours.loc[user]
    global predict_rate 
    predict_rate = weighted_ratings / sum_similarity
  return predict_rate.values[0]

In [ ]:
start = time.time()
print("Prediction results is: {:.2f} star".format(predict_rating(7, 1)))
end = time.time()
print("Simple Recommendation task completed in {:.4f} seconds" .format(end-start))

Prediction results is: 4.26 star
Simple Recommendation task completed in 0.0320 seconds


In [ ]:
user7_movies_set = ratings_df[ratings_df['userId'] == 7]
user7_movies_set

,userId,movieId,rating
874,7,1,4.5
875,7,50,4.5
876,7,58,3.0
877,7,150,4.5
878,7,165,4.0
...,...,...,...
1021,7,48997,2.5
1022,7,49272,4.5
1023,7,49278,3.5
1024,7,49286,0.5


In [ ]:
user7_not_rated = movies_df.set_index('movieId').subtract(user7_movies_set.set_index('movieId'), axis='columns')
user7_not_rated 

,rating,title,userId
movieId,,,
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
...,...,...,...
193581,NaN,NaN,NaN
193583,NaN,NaN,NaN
193585,NaN,NaN,NaN


In [ ]:
recommendations =[]
for movie in user7_not_rated.index:
  prediction = predict_rating(7, movie)
  if prediction >= 4.5:
    recommendations.append(movie)

In [ ]:
print(len(recommendations))
recommendations[0:5]

802


[40, 50, 53, 99, 123]


1.   Đối chiếu với moviesDataFrame.
2.   Tính độ popular của movie = lượt rating.
3.   Filter recommendations vs threshold popular > mean(popular)

